In [2]:
import pandas as pd
import numpy as np
import os
from scipy import stats
from glob import glob
from tqdm import tqdm

In [3]:
BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/HAR_Research_Project/data/raw'  # 원본 데이터가 있는 최상위 폴더
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/HAR_Research_Project/data/processed'  # 전처리된 .npy 파일을 저장할 폴더

# 저장 폴더가 없으면 생성
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# 공통 하이퍼파라미터
TIME_STEPS = 128
STEP = 64  # Overlap 50% (128의 절반)

In [4]:
# ==========================================
# [유틸리티] 슬라이딩 윈도우 생성 함수
# 연속된 시계열 데이터를 모델에 넣기 좋게 (Samples, 128, Channels)로 자름
# ==========================================
def create_segments(data, labels, time_steps, step):
    segments = []
    labels_list = []

    # 데이터 길이가 윈도우보다 짧으면 패스
    if len(data) < time_steps:
        return np.empty((0, time_steps, data.shape[1])), np.empty((0,))

    # Step만큼 이동하며 윈도우 자르기
    for i in range(0, len(data) - time_steps, step):
        xs = data[i : i + time_steps]
        # 해당 윈도우에서 가장 많이 등장한 라벨(Mode)을 정답으로 채택
        ys = stats.mode(labels[i : i + time_steps], keepdims=True)[0][0]
        segments.append(xs)
        labels_list.append(ys)

    return np.array(segments), np.array(labels_list)

In [5]:
# ==========================================
# 1. UCI-HAR 전처리
# 특징: 이미 128로 세그먼트되어 있음. txt 파일을 읽어서 합치기만 하면 됨.
# ==========================================
def process_uci_har():
    print("\n[1/5] Processing UCI-HAR...")
    dataset_path = os.path.join(BASE_PATH, 'UCI_HAR_Dataset')

    if not os.path.exists(dataset_path):
        print("❌ Skipping UCI-HAR (Folder not found)")
        return

    # 읽어올 신호 파일 목록 (총 9채널: Body Acc, Body Gyro, Total Acc)
    # 순서: train/test 폴더 안의 Inertial Signals 폴더에 있음
    signal_files = [
        'body_acc_x_', 'body_acc_y_', 'body_acc_z_',
        'body_gyro_x_', 'body_gyro_y_', 'body_gyro_z_',
        'total_acc_x_', 'total_acc_y_', 'total_acc_z_'
    ]

    def load_group(group):
        # group: 'train' or 'test'
        signals_data = []
        path = os.path.join(dataset_path, group, 'Inertial Signals')

        for file_name in signal_files:
            file_path = os.path.join(path, f"{file_name}{group}.txt")
            # 공백으로 구분된 텍스트 파일 읽기
            signals_data.append(
                pd.read_csv(file_path, delim_whitespace=True, header=None).values
            )

        # (Channels, Samples, 128) -> (Samples, 128, Channels)로 변환
        # np.dstack은 깊이(3번째 차원) 방향으로 쌓음
        return np.dstack(signals_data)

    def load_y(group):
        file_path = os.path.join(dataset_path, group, f"y_{group}.txt")
        return pd.read_csv(file_path, delim_whitespace=True, header=None).values.flatten() - 1 # 0-based index

    try:
        # 합치지 않고 각각 로드 및 저장
        X_train = load_group('train')
        y_train = load_y('train')
        X_test = load_group('test')
        y_test = load_y('test')

        # Train / Test 별도 저장
        np.save(f'{SAVE_PATH}/UCI_X_train.npy', X_train)
        np.save(f'{SAVE_PATH}/UCI_y_train.npy', y_train)
        np.save(f'{SAVE_PATH}/UCI_X_test.npy', X_test)
        np.save(f'{SAVE_PATH}/UCI_y_test.npy', y_test)

        print(f"✅ UCI-HAR Done (Fixed Split).")
        print(f"   Train: {X_train.shape}, Test: {X_test.shape}")

    except Exception as e:
        print(f"❌ Error UCI-HAR: {e}")

In [ ]:
# ==========================================
# 2. WISDM 전처리
# 특징: 텍스트 파일 끝에 세미콜론(;)이 있어 파싱 에러 발생 -> 제거 필요
# ==========================================
def process_wisdm():
    print("\n[2/5] Processing WISDM...")
    file_path = os.path.join(BASE_PATH, 'WISDM_ar_v1.1', 'WISDM_ar_v1.1_raw.txt')

    if not os.path.exists(file_path):
        print(f"❌ Skipping WISDM (File not found)")
        return

    columns = ['user', 'activity', 'timestamp', 'x', 'y', 'z']

    try:
        # on_bad_lines='skip': 형식이 잘못된 라인은 건너뜀
        df = pd.read_csv(file_path, header=None, names=columns, on_bad_lines='skip')

        # [핵심] Z축 데이터 뒤에 붙은 ';' 제거하고 실수형으로 변환
        df['z'] = df['z'].astype(str).str.replace(';', '', regex=False).astype(float)
        df.dropna(inplace=True)

        # 문자열 라벨(Jogging 등)을 숫자로 변환
        activities = df['activity'].unique()
        label_map = {label: i for i, label in enumerate(activities)}
        df['activity_id'] = df['activity'].map(label_map)

        # 슬라이딩 윈도우 적용
        X, y = create_segments(df[['x', 'y', 'z']].values, df['activity_id'].values, TIME_STEPS, STEP)

        np.save(f'{SAVE_PATH}/WISDM_X.npy', X)
        np.save(f'{SAVE_PATH}/WISDM_y.npy', y)
        print(f"✅ WISDM Done. Shape: {X.shape}, Classes: {len(activities)}")

    except Exception as e:
        print(f"❌ Error WISDM: {e}")

In [ ]:
# ==========================================
# 3. UniMiB SHAR 전처리
# 특징: Matlab 데이터가 평탄화되어 있거나 축이 꼬여 있음. (N, 3, 151) -> (N, 128, 3)
# ==========================================
def process_unimib():
    print("\n[3/5] Processing UniMiB SHAR (Direct Access Mode)...")
    file_path = os.path.join(BASE_PATH, 'UniMiB_SHAR_Dataset', 'full_data.npy')

    if not os.path.exists(file_path):
        print(f"❌ Skipping UniMiB (full_data.npy not found)")
        return

    try:
        # 1. 데이터 로드
        full_data = np.load(file_path, allow_pickle=True)
        # (30, 5) -> 150 items (각 피험자별 실험 데이터)
        flat_data = full_data.flatten()

        class_names = [
            'Walking', 'Running', 'GoingUpS', 'GoingDownS', 'Jumping',
            'SittingDown', 'StandingUpFS', 'LyingDownFS', 'StandingUpFL',
            'FallingForw', 'FallingBack', 'FallingRight', 'FallingLeft',
            'Syncope', 'FallingBackSC', 'FallingWithPS', 'HittingObstacle'
        ]

        X_list = []
        y_list = []

        # 2. 데이터 순회 (총 150개)
        for item_index, item_wrapper in enumerate(flat_data):
            # [Step 1] (1, 1) 껍질 강제 해제
            # 대부분의 데이터가 (1, 1) 안에 숨어 있음
            try:
                if isinstance(item_wrapper, np.ndarray) and item_wrapper.shape == (1, 1):
                    item = item_wrapper[0, 0]
                else:
                    item = item_wrapper
            except:
                continue

            # [Step 2] 구조체 필드 확인
            if not (hasattr(item, 'dtype') and item.dtype.names):
                continue

            # [Step 3] 각 클래스별 데이터 추출
            for label_id, name in enumerate(class_names):
                if name in item.dtype.names:
                    # 'Walking' 데이터 꺼내기 (예상: (2, 1) 배열)
                    container = item[name]

                    # (2, 1) 형태의 컨테이너 내부 탐색
                    if isinstance(container, np.ndarray):
                        # flatten()으로 (2, 1) -> [요소1, 요소2] 로 만들어서 전수 조사
                        sub_items = container.flatten()

                        for real_data in sub_items:
                            # [핵심] 데이터가 유효한지 확인 (Shape (6, N))
                            if isinstance(real_data, np.ndarray) and real_data.ndim == 2:
                                if real_data.shape[0] == 6:
                                    # 데이터 발견! (가속도 X, Y, Z는 0, 1, 2행)
                                    # (6, Time) -> (3, Time) -> (Time, 3)
                                    raw_signal = real_data[:3, :].T

                                    # 슬라이딩 윈도우 적용
                                    dummy_labels = np.full(len(raw_signal), label_id)
                                    xs, ys = create_segments(raw_signal, dummy_labels, TIME_STEPS, STEP)

                                    if len(xs) > 0:
                                        X_list.append(xs)
                                        y_list.append(ys)

        # 4. 병합 및 저장
        if X_list:
            X = np.concatenate(X_list)
            y = np.concatenate(y_list)

            np.save(f'{SAVE_PATH}/UniMiB_X.npy', X)
            np.save(f'{SAVE_PATH}/UniMiB_y.npy', y)
            print(f"✅ UniMiB Done. Shape: {X.shape}, Classes: {len(np.unique(y))} (Target: 17)")
        else:
            print("❌ Error: No segments created. (Data list is empty)")
            # 디버깅을 위해 첫 번째 아이템의 상태 출력
            print("DEBUG: First item structure check failed.")

    except Exception as e:
        print(f"❌ Critical Error UniMiB: {e}")

In [ ]:
# ==========================================
# 4. PAMAP2 전처리
# 특징: 결측치(NaN)가 많음 -> 보간법 사용. 샘플링 레이트가 높음(100Hz) -> 다운샘플링.
# ==========================================
def process_pamap2():
    print("\n[4/5] Processing PAMAP2...")
    proto_path = os.path.join(BASE_PATH, 'PAMAP2_Dataset', 'Protocol')

    if not os.path.exists(proto_path):
        print("❌ Skipping PAMAP2 (Folder not found)")
        return

    # 사용할 컬럼 인덱스 (총 9채널 가속도 + 1개 라벨)
    # Col 1: ActivityID
    # Col 4-6: Hand Acc, 21-23: Chest Acc, 38-40: Ankle Acc
    relevant_cols = [1, 4, 5, 6, 21, 22, 23, 38, 39, 40]

    data_collection = []
    label_collection = []

    # subject101.dat ~ subject109.dat 파일 찾기
    files = glob(os.path.join(proto_path, '*.dat'))

    for f in tqdm(files, desc="Reading PAMAP2 Files"):
        try:
            df = pd.read_csv(f, sep=' ', header=None)
            df = df.iloc[:, relevant_cols]

            # [핵심] 결측치 선형 보간 (Interpolation)
            df = df.interpolate(method='linear', limit_direction='both')
            df.dropna(inplace=True)

            # [핵심] 다운샘플링 (100Hz -> ~33Hz)
            # 3개 중 1개만 선택하여 데이터 크기 줄임
            df = df.iloc[::3, :]

            # Activity ID 0 (Transient/Null) 제거
            df = df[df[1] != 0]

            # 데이터 분리
            data = df.iloc[:, 1:].values # 9 Channels
            labels = df.iloc[:, 0].values

            # 세그먼트 생성
            X_sub, y_sub = create_segments(data, labels, TIME_STEPS, STEP)

            if len(X_sub) > 0:
                data_collection.append(X_sub)
                label_collection.append(y_sub)

        except Exception as e:
            print(f"Error reading {os.path.basename(f)}: {e}")

    if data_collection:
        X = np.concatenate(data_collection)
        y = np.concatenate(label_collection)

        # 라벨 리매핑 (불연속적인 ID들을 0부터 차례대로 매핑)
        unique_labels = np.unique(y)
        label_map = {original: new for new, original in enumerate(unique_labels)}
        y = np.vectorize(label_map.get)(y)

        np.save(f'{SAVE_PATH}/PAMAP2_X.npy', X)
        np.save(f'{SAVE_PATH}/PAMAP2_y.npy', y)
        print(f"✅ PAMAP2 Done. Shape: {X.shape}, Classes: {len(unique_labels)}")
    else:
        print("❌ PAMAP2: No data processed.")

In [ ]:
# ==========================================
# 5. Opportunity 전처리
# 특징: 센서가 매우 많고 복잡함. 필요한 센서만 선별하고 Null Class 제거 필요.
# ==========================================
def process_opportunity():
    print("\n[5/5] Processing Opportunity...")
    data_dir = os.path.join(BASE_PATH, 'OpportunityUCIDataset')

    # 'ADL' (일상 행동) 파일만 재귀적으로 검색
    files = glob(os.path.join(data_dir, '**', '*.dat'), recursive=True)
    files = [f for f in files if 'ADL' in os.path.basename(f)]

    if not files:
        print("❌ Skipping Opportunity (No ADL files found)")
        return

    # 사용할 컬럼 인덱스 (Opportunity 문서 및 데이터 정찰 결과 기반)
    # Back Acc: 37,38,39 / RUA Acc: 50,51,52 / Label(Locomotion): 243
    feature_cols = [37, 38, 39, 50, 51, 52]
    label_col = 243
    use_cols = feature_cols + [label_col]

    data_collection = []
    label_collection = []

    for f in tqdm(files, desc="Reading Opportunity Files"):
        try:
            # 공백 구분, 필요한 컬럼만 로드하여 메모리 절약
            df = pd.read_csv(f, header=None, delim_whitespace=True, usecols=use_cols)

            # 결측치 보간
            df = df.interpolate(method='linear', limit_direction='both')
            df.dropna(inplace=True)

            # 컬럼 순서 보장 (Features..., Label)
            df = df[feature_cols + [label_col]]

            # Null Class(0) 제거 (의미 없는 구간)
            df = df[df[label_col] != 0]

            data = df.iloc[:, :-1].values
            labels = df.iloc[:, -1].values

            X_sub, y_sub = create_segments(data, labels, TIME_STEPS, STEP)

            if len(X_sub) > 0:
                data_collection.append(X_sub)
                label_collection.append(y_sub)

        except Exception as e:
            print(f"Error processing {os.path.basename(f)}: {e}")

    if data_collection:
        X = np.concatenate(data_collection)
        y = np.concatenate(label_collection)

        # 라벨 리매핑 (1:Stand, 2:Walk, 4:Sit, 5:Lie -> 0,1,2,3)
        unique_labels = sorted(np.unique(y))
        label_map = {original: new for new, original in enumerate(unique_labels)}
        y = np.vectorize(label_map.get)(y)

        np.save(f'{SAVE_PATH}/Opportunity_X.npy', X)
        np.save(f'{SAVE_PATH}/Opportunity_y.npy', y)
        print(f"✅ Opportunity Done. Shape: {X.shape}, Classes: {len(unique_labels)}")
    else:
        print("❌ Opportunity: No data processed.")

In [6]:
# ==========================================
# 메인 실행부
# ==========================================
if __name__ == "__main__":
    print(f"🚀 Starting Data Preprocessing...")
    print(f"📂 Base Path: {BASE_PATH}")
    print(f"💾 Save Path: {SAVE_PATH}")

    process_uci_har()
    process_wisdm()
    process_unimib()
    process_pamap2()
    process_opportunity()

    print("\n🎉 All Preprocessing Steps Finished!")

🚀 Starting Data Preprocessing...
📂 Base Path: /content/drive/MyDrive/Colab Notebooks/HAR_Research_Project/data/raw
💾 Save Path: /content/drive/MyDrive/Colab Notebooks/HAR_Research_Project/data/processed

[1/5] Processing UCI-HAR...


/tmp/ipython-input-118896812.py:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  pd.read_csv(file_path, delim_whitespace=True, header=None).values
/tmp/ipython-input-118896812.py:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  pd.read_csv(file_path, delim_whitespace=True, header=None).values
/tmp/ipython-input-118896812.py:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  pd.read_csv(file_path, delim_whitespace=True, header=None).values
/tmp/ipython-input-118896812.py:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  pd.read_csv(file_path, delim_whitespace=True, header=None).values
/tmp/ipython-inp

✅ UCI-HAR Done (Fixed Split).
   Train: (7352, 128, 9), Test: (2947, 128, 9)

🎉 All Preprocessing Steps Finished!
